In [66]:
import pandas as pd 
import numpy as np
import geopandas as gpd 
import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
import time
import seaborn as sns
import ast
import requests
from time import sleep
import openrouteservice
from openrouteservice import client
from openrouteservice import convert
from shapely.geometry import shape, Point
from descartes import PolygonPatch
import network_scan_helper as ns
import importlib
importlib.reload(ns)
import contextily as cx
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

import logging
# Clear any existing handlers to avoid duplicate logs
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,  # Log everything from DEBUG and above
    format='%(asctime)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.FileHandler("log.txt"),  # Save to file
        logging.StreamHandler()          # Show in notebook
    ]
)




# import administrative boundaries

administrative boundary was downloaded from: 
https://data.humdata.org/dataset/cod-ab-phl

### 🇵🇭 Administrative Structure of the Philippines

| Level | Administrative Division     | Description / International Equivalent                                         | Number (Approx.)                      |
|-------|-----------------------------|--------------------------------------------------------------------------------|---------------------------------------|
| 1     | **Country**                 | Sovereign state                                                                | 1                                     |
| 2     | **Region**                  | Grouping of provinces for administration and planning (limited autonomy)       | 17 (including BARMM)                  |
| 3     | **Province**                | Major political-administrative division; like a state or county               | 82                                    |
| 4     | **City / Municipality**     | Subdivisions of provinces; cities are more autonomous than municipalities      | ~1,634 (149 cities, 1,485 municipalities) |
| 5     | **Barangay**                | Smallest official unit; like a village, neighborhood, or ward                  | ~42,000+                              |

#### 🔍 Notes:
- **Unit of analysis** we are using adm level 4: City/Municipality as a unit of analysis, i.e., City of Manila, Quezon City, San Miguel, Mangatarem, etc. However, in the file downloaded it is labeled ADM3.
- **Regions** are primarily administrative and don’t have elected governments, *except* for **BARMM** (Bangsamoro Autonomous Region in Muslim Mindanao), which is autonomous.
- **Cities** can be:
  - Highly Urbanized Cities (independent of provinces)
  - Component Cities (within provinces)
  - Independent Component Cities (not under provincial control)
- **Barangays** are crucial for community-level governance and data collection.


- focus on road infrastructure investment
- redefining city definition


In [15]:
# import basic administrative data for the whole country
gdf = gpd.read_file('0_raw_data/administrative boundary/phl_admbnda_adm3_psa_namria_20231106.shp')


# Run analysis for list of cities

In [ ]:
importlib.reload(ns)

# define city
city_list = list(gdf[gdf.ADM1_EN=='National Capital Region (NCR)']['ADM3_EN']) # trying all cities within National Capital Region 
proj_crs = 3121


summary_stats = []
for city in city_list:    
    try:
        # Suppress DeprecationWarnings within this block
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            # START
            start = time.time()
            # --------------------------------------------
            # 1. making AOI
            logging.info(f'\n\n ------computing for {city}------')
            logging.info(f'1. making AOI for {city}')
            gdf_city, gdf_city_buffer = ns.get_AOI(gdf=gdf, city=city, proj_crs=proj_crs)
            logging.info(f"done in {time.time() - start:.2f} seconds")
            
            # 2. extracting graph, edges, nodes
            logging.info(f'2. extracting graph, edges, nodes for {city}')
            road_graph, road_edges, road_nodes = ns.extract_road_network(gdf_polygon=gdf_city_buffer, network_type='all', simplify=True, verbose=False)
            # export to geojson
            road_edges_export = ns.sanitize_gdf_for_export(road_edges)
            road_edges_export.to_file(f'output/{city}/geojson/road_edges.geojson')
            road_nodes_export = ns.sanitize_gdf_for_export(road_nodes)
            road_nodes_export.to_file(f'output/{city}/geojson/road_nodes.geojson')
            logging.info(f"done in {time.time() - start:.2f} seconds")
            
            # 3. making basemap
            logging.info(f'3. generating basemap AOI for {city}')
            fig,ax = ns.map_basemap(gdf_city=gdf_city, gdf_city_buffer=gdf_city_buffer, road_edges=road_edges, road_nodes=road_nodes,crs=proj_crs)
            fig.savefig(f'output/{city}/png/map_AOI.png', dpi=300, bbox_inches = 'tight')
            logging.info(f"done in {time.time() - start:.2f} seconds")

            # 4. plotting road hierarchy
            logging.info(f'4. plotting road hierarchy for {city}')
            fig, ax = ns.map_road_hierarchy(road_edges=road_edges.clip(gdf_city), gdf_border=gdf_city, crs=3121, colormap='plasma')
            fig.savefig(f'output/{city}/png/map_road_hierarchy.png', dpi=300, bbox_inches = 'tight')
            logging.info(f"done in {time.time() - start:.2f} seconds")

            # 5. road hierarchy analysis
            # calculate length
            logging.info(f'5. analyzing road hierarchy for {city}')
            desc = road_edges.groupby('highway_cat')['length'].describe()
            desc['sum'] = road_edges.groupby('highway_cat')['length'].sum()
            desc['length_km'] = desc['sum']/1000
            desc.to_csv(f'output/{city}/tabular/road_by_hierarchy.csv')
            fig, ax = ns.plot_total_length_by_cat(desc=desc)
            fig.savefig(f'output/{city}/png/chart_totalroadlength_by_category.png', dpi=300, bbox_inches='tight')
            fig, ax = ns.plot_median_length_by_cat(desc=desc)
            fig.savefig(f'output/{city}/png/chart_medianroadlength_by_category.png', dpi=300, bbox_inches='tight')
            logging.info(f"done in {time.time() - start:.2f} seconds")

            # 6. road orientations
            logging.info(f'6. mapping road orientation for {city}')
            road_bearing = ox.add_edge_bearings(ox.convert.to_undirected(road_graph))
            fig, ax = ox.plot.plot_orientation(road_bearing, title=city, area=True, figsize=(8,8))
            plt.close()
            fig.savefig(f'output/{city}/png/chart_road_orientation.png', dpi=300, bbox_inches='tight')
            logging.info(f"done in {time.time() - start:.2f} seconds")

            # 7. calculate basic stats
            logging.info(f'7. calculating basic stats for {city}')
            df_stats = ns.calc_basic_stats(graph=road_graph)
            df_stats.to_csv(f'output/{city}/tabular/basic_stats.csv')
            stats_dict = df_stats[['Metric', 'Value']].set_index('Metric')['Value'].to_dict()
            logging.info(f"done in {time.time() - start:.2f} seconds")

            # 8. calculate lanes and maxspeed
            logging.info(f'8. calculating median lanes and maxspeed for {city}')
            lanes_speed_df, lanes_sort, speed_sort = ns.compute_lanes_maxspeed(road_edges=road_edges)
            lanes_speed_df.to_csv(f'output/{city}/tabular/lanes_speed_by_hierarchy.csv')
            fig, ax = ns.plot_median_lanes(lanes_sort=lanes_sort)
            fig.savefig(f'output/{city}/png/lanes_by_hierarchy.png', dpi=300, bbox_inches = 'tight')
            fig, ax = ns.plot_median_speed(speed_sort=speed_sort)
            fig.savefig(f'output/{city}/png/speed_by_hierarchy.png', dpi=300, bbox_inches = 'tight')
            logging.info(f"done in {time.time() - start:.2f} seconds")
            
            # 9. calculate and visualize intersection/node degree
            logging.info(f'9. calculate and visualize intersection/node degree for {city}')
            street_node_df = ns.calculate_intersection(df_stats=df_stats)
            street_node_df.to_csv(f'output/{city}/tabular/intersection_by_degree_counts.csv')
            fig,ax = ns.map_road_intersections(road_edges=road_edges.clip(gdf_city), road_nodes=road_nodes.clip(gdf_city), gdf_border=gdf_city, crs=3121, colormap='magma_r')
            fig.savefig(f'output/{city}/png/map_road_intersections.png', dpi=300, bbox_inches = 'tight')
            logging.info(f"done in {time.time() - start:.2f} seconds")

            # 10. computing and mapping centrality analysis
            logging.info(f'10. centrality analysis for {city}')
            # determining sample size 5% of nodes, bounded between 100–1000
            n_nodes = len(road_nodes)
            k = min(1000, max(100, int(n_nodes * 0.05)))
            logging.info(f'total nodes: {n_nodes}, sample size (k): {k}')
            # compute centrality
            nodes_centrality, edges_centrality =ns.compute_graph_centralities(graph=road_graph, 
                                                                            degree_centrality=True, 
                                                                            node_closeness_centrality=True, 
                                                                            node_betweenness_centrality=True,
                                                                            edge_betweenness_centrality=True,
                                                                            k=k, #scale according to city size, alternatively use 500 to save computing time
                                                                            seed=16
                                                                            )
            # map closeness centrality
            fig, ax = ns.map_nodes_centrality(road_edges=edges_centrality.clip(gdf_city), road_nodes=nodes_centrality.clip(gdf_city), gdf_border=gdf_city, crs=3121, colormap='RdYlGn', column='closeness_centrality')
            fig.savefig(f'output/{city}/png/map_closeness_centrality.png', dpi=300, bbox_inches='tight' ) # figure out ways for different weighting, i.e., decay values?
            # map betweenness centrality
            fig, ax = ns.map_nodes_centrality(road_edges=edges_centrality.clip(gdf_city), road_nodes=nodes_centrality.clip(gdf_city), gdf_border=gdf_city, crs=3121, colormap='magma', column='betweenness_centrality')
            fig.savefig(f'output/{city}/png/map_nodes_betweenness_centrality.png', dpi=300, bbox_inches='tight' )
            # map edges betweenness centrality
            fig, ax = ns.map_edges_centrality(road_edges=edges_centrality.clip(gdf_city), road_nodes=nodes_centrality.clip(gdf_city), gdf_border=gdf_city, crs=3121, colormap='magma')
            fig.savefig(f'output/{city}/png/map_edges_betweenness_centrality.png', dpi=300, bbox_inches='tight' )
            # map degree centrality
            fig, ax = ns.map_nodes_centrality(road_edges=edges_centrality.clip(gdf_city), road_nodes=nodes_centrality.clip(gdf_city), gdf_border=gdf_city, crs=3121, colormap='RdYlBu_r', column='degree_centrality')
            fig.savefig(f'output/{city}/png/map_degree_centrality.png', dpi=300, bbox_inches='tight' )
            # export to geojson
            nodes_centrality_export = ns.sanitize_gdf_for_export(nodes_centrality)
            nodes_centrality_export.to_file(f'output/{city}/geojson/nodes_centrality.geojson')
            edges_centrality_export = ns.sanitize_gdf_for_export(edges_centrality)
            edges_centrality_export.to_file(f'output/{city}/geojson/edges_centrality.geojson')
            logging.info(f"done in {time.time() - start:.2f} seconds")

            # 11. accessibility analysis
            logging.info(f'11. accessibility analysis for {city}')
            # education facilities
            ns.analyze_accessibility(
                category='education',
                tags={'amenity': ['school', 'kindergarten', 'college', 'university', 'library']},
                road_graph=road_graph,
                gdf_city_buffer=gdf_city_buffer,
                gdf_city=gdf_city,
                city=city, 
                interpolation=False
            )
            
            # healthcare facilities
            ns.analyze_accessibility(
                category='healthcare',
                tags={'amenity': ['clinic', 'hospital']},
                road_graph=road_graph,
                gdf_city_buffer=gdf_city_buffer,
                gdf_city=gdf_city,
                city=city, 
                interpolation=False
            )
            
            # transport facilities
            ns.analyze_accessibility(
                category='transport',
                tags = {
                    'amenity': ['bus_station'],
                    'public_transport': ['station', 'platform'],
                    'railway': ['station'],
                },
                road_graph=road_graph,
                gdf_city_buffer=gdf_city_buffer,
                gdf_city=gdf_city,
                city=city, 
                interpolation=False
            )
            
            # green open space facilities
            ns.analyze_accessibility(
                category='green',
                tags = {
                    'leisure': ['park', 'garden', 'playground', 'pitch'],
                    'landuse': ['recreation_ground', 'grass'],
                    'natural': ['wood', 'scrub'],
                    'place': ['square']
                },
                road_graph=road_graph,
                gdf_city_buffer=gdf_city_buffer,
                gdf_city=gdf_city,
                city=city, 
                interpolation=True
            )
            
            # fire station facilities
            ns.analyze_accessibility(
                category='fire',
                tags = {
                    'amenity': ['fire_station'],
                },
                road_graph=road_graph,
                gdf_city_buffer=gdf_city_buffer,
                gdf_city=gdf_city,
                city=city, 
                interpolation=False
            )
            
            # food supplies facilities
            ns.analyze_accessibility(
                category='food',
                tags = {
                    'shop': ['supermarket', 'convenience', 'marketplace'], 
                    'amenity': 'marketplace',
                },
                road_graph=road_graph,
                gdf_city_buffer=gdf_city_buffer,
                gdf_city=gdf_city,
                city=city, 
                interpolation=False
            )


            # --------------------------------------------
            # END
            end = time.time()
            total_seconds = int(end - start)
            minutes, seconds = divmod(total_seconds, 60)
            duration_str = f'{minutes} min {seconds} sec'

            # --- store combined results ---
            stats_dict['City'] = city
            stats_dict['Province'] = gdf_city.ADM3_EN.values[0]
            stats_dict['Region'] = gdf_city.ADM2_EN.values[0]
            
            stats_dict['AREA_SQKM'] = gdf_city.AREA_SQKM.values[0]
            stats_dict['duration'] = duration_str
            summary_stats.append(stats_dict)
    
    except Exception as e: 
        logging.info(f"⚠️ Skipping {city['name']} due to error: {e}")
        continue

# Convert list of dicts to a DataFrame
summary_df = pd.DataFrame(summary_stats)

# Move 'city' and 'duration' to front
cols = ['City', 'Province', 'Region'] + [col for col in summary_df.columns if col not in ['City', 'Region', 'Province', 'duration']] + ['duration']
summary_df = summary_df[cols]
# Create timestamp string
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
summary_df.to_csv(f'output/{timestamp}_summary_statistics.csv')
display(summary_df)


